## Libraries

In [1]:
# Common:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
import resampy
import struct

# Features engineering:
from python_speech_features import mfcc, delta

# Keras tools:
import tensorflow as tf
import keras

# Sklearn tools:
from sklearn import model_selection, pipeline, metrics

# Graphics:
import matplotlib.pyplot as plt

## Common params and settings

In [2]:
# Paths:
TRAIN_FILE = 'C:/Users/Alex/Desktop/Untitled Folder/Features/train_features.csv' # Path to file with train data
REAL_SIGNAL = 'C:/Users/Alex/Desktop/Vega-VAD project/data/real/room.wav' # Test wav file

# Saved models:
models_inst = {
    'v1': 'models/DNN_model_v1.h5', # {128, 64, 32, 16, 1}
    'v2': 'models/DNN_model_v2.h5', # {128, 1}
    'v3': 'models/DNN_model_v3.h5', # {64, 1}
    'v4': 'models/DNN_model_v4.h5', # {32, 32, 1}
    'v5': 'models/DNN_model_v5.h5', # {32, 1}
    'v6': 'models/DNN_model_v6.h5', # {32, 16, 1}
    'v7': 'models/DNN_model_v7.h5', # {128, 128, 128}
    'v8': 'models/DNN_model_v8.h5', # {32, 32, 32, 1}
}

structure_info = {
    'v1': 'DNN: {128, 64, 32, 16, 1}',
    'v2': 'DNN: {128, 1}',
    'v3': 'DNN: {64, 1}',
    'v4': 'DNN: {32, 32, 1}',
    'v5': 'DNN: {32, 1}',
    'v6': 'DNN: {32, 16, 1}',
    'v7': 'DNN: {128, 128, 128, 1}',
    'v8': 'DNN: {32, 32, 32, 1}'
}

current_version = 'v3'
TRAIN_MODE = True

# Features:
SAMPLE_RATE = 16000
PREEMPHASIS_COEF = 0.95  # Coefficient for pre-processing filter
FRAME_LENGTH = 0.032  # Window length in sec
FRAME_STEP = 0.01  # Length of step in sec
WINDOW_FUNCTION = np.hamming  # Type of window function
NFFT = 512  # Points of FFT
NFILTERS = 26 # Filters number for filterbanks transformation
CEPLIFTER = 22 # Lifter coefficent
APPEND_ENERGY = True # Append energy to mfcc (Append or not)
NFEATURES = 13  # Number of Mel filters

# Params for MLP classifier learning:
CHUNK_SIZE = 10 ** 6 # Sample number in one series
TEST_SIZE = 0.3 # Ration Test/Train size
LAYERS = [64] # Neurons number in the layers
ACTIVATION_FUNCTION = 'relu' # Function at the end of the layers
OUTPUT_ACTIVATION_FUNCTION = 'sigmoid' # Activation function at the final layer
SOLVER = 'adam' # Optimizer
LOSS_FUNCTION = 'binary_crossentropy' # loss-function
METRIC = ['accuracy'] # Metrics for learning
BATCH_SIZE = 64 # Bactch size
EPOCHS = 25 # Number of the learning cycles
CALL_BACKS = None # Info

## Building and fitting the model

In [3]:
def createModel() -> keras.engine.sequential.Sequential:
    model = tf.keras.models.Sequential()
    model.add(tf.keras.Input(shape=(NFEATURES * 3, )))
    model.add(tf.keras.layers.BatchNormalization())
    for i in range(len(LAYERS)):
        model.add(tf.keras.layers.Dense(LAYERS[i], activation=ACTIVATION_FUNCTION))
    model.add(tf.keras.layers.Dense(1, activation=OUTPUT_ACTIVATION_FUNCTION))
    model.compile(optimizer=SOLVER, loss=LOSS_FUNCTION, metrics=METRIC)
    
    return model

In [4]:
def loadModel(model_version: str) -> keras.engine.sequential.Sequential:
    model = tf.keras.models.load_model(model_version)
    
    return model

In [5]:
def learnModel(model_version: str) -> None:
    first_run = True
    # Load data set for training:
    for chunk in pd.read_csv(TRAIN_FILE, chunksize=CHUNK_SIZE):
        dataset_chunk = chunk.to_numpy(dtype=np.float32, copy=False)
        labels = dataset_chunk[:, 0]
        data_feat = dataset_chunk[:, 1:]
    
        # Splitting data:
        X_train, X_test, y_train, y_test = model_selection.train_test_split(data_feat,
                                                                            labels,
                                                                            test_size=TEST_SIZE,
                                                                            shuffle=True,
                                                                            random_state=None)

        if not first_run:
            model = tf.keras.models.load_model(model_version)
        else:
            model = createModel()
            first_run = False
    
        model.fit(X_train,
                  y_train,
                  validation_data=(X_test, y_test),
                  epochs=EPOCHS,
                  batch_size=BATCH_SIZE,
                  callbacks=CALL_BACKS)
    
        model.save(model_version)

In [ ]:
if TRAIN_MODE:
    learnModel(models_inst[current_version])

# If you want to load the model:
model = loadModel(models_inst[current_version])

Epoch 1/25
10938/10938 [==============================] - 11s 906us/step - loss: 0.3718 - accuracy: 0.8282 - val_loss: 0.3406 - val_accuracy: 0.8460
Epoch 2/25
10938/10938 [==============================] - 9s 847us/step - loss: 0.3451 - accuracy: 0.8423 - val_loss: 0.3298 - val_accuracy: 0.8498
Epoch 3/25
10938/10938 [==============================] - 9s 846us/step - loss: 0.3372 - accuracy: 0.8462 - val_loss: 0.3254 - val_accuracy: 0.8547
Epoch 4/25
10938/10938 [==============================] - 10s 896us/step - loss: 0.3318 - accuracy: 0.8487 - val_loss: 0.3217 - val_accuracy: 0.8576
Epoch 5/25
10938/10938 [==============================] - 9s 851us/step - loss: 0.3278 - accuracy: 0.8512 - val_loss: 0.3204 - val_accuracy: 0.8587
Epoch 6/25
10938/10938 [==============================] - 9s 851us/step - loss: 0.3247 - accuracy: 0.8523 - val_loss: 0.3122 - val_accuracy: 0.8577
Epoch 7/25
10938/10938 [==============================] - 10s 880us/step - loss: 0.3230 - accuracy: 0.8532 - v

10938/10938 [==============================] - 9s 838us/step - loss: 0.3045 - accuracy: 0.8649 - val_loss: 0.2946 - val_accuracy: 0.8715
Epoch 7/25
10938/10938 [==============================] - 9s 849us/step - loss: 0.3045 - accuracy: 0.8648 - val_loss: 0.2928 - val_accuracy: 0.8686
Epoch 8/25
10938/10938 [==============================] - 9s 841us/step - loss: 0.3039 - accuracy: 0.8645 - val_loss: 0.2959 - val_accuracy: 0.8662
Epoch 9/25
10938/10938 [==============================] - 9s 847us/step - loss: 0.3032 - accuracy: 0.8652 - val_loss: 0.2929 - val_accuracy: 0.8696
Epoch 10/25
10938/10938 [==============================] - 9s 850us/step - loss: 0.3028 - accuracy: 0.8654 - val_loss: 0.2926 - val_accuracy: 0.8690
Epoch 11/25
10938/10938 [==============================] - 10s 878us/step - loss: 0.3028 - accuracy: 0.8657 - val_loss: 0.2919 - val_accuracy: 0.8707
Epoch 12/25
10938/10938 [==============================] - 9s 854us/step - loss: 0.3028 - accuracy: 0.8652 - val_loss: 0

10938/10938 [==============================] - 9s 846us/step - loss: 0.3141 - accuracy: 0.8594 - val_loss: 0.3010 - val_accuracy: 0.8661
Epoch 12/25
10938/10938 [==============================] - 9s 835us/step - loss: 0.3142 - accuracy: 0.8593 - val_loss: 0.3013 - val_accuracy: 0.8652
Epoch 13/25
10938/10938 [==============================] - 9s 846us/step - loss: 0.3134 - accuracy: 0.8595 - val_loss: 0.3034 - val_accuracy: 0.8657
Epoch 14/25
10938/10938 [==============================] - 9s 840us/step - loss: 0.3134 - accuracy: 0.8597 - val_loss: 0.3033 - val_accuracy: 0.8637
Epoch 15/25
10938/10938 [==============================] - 9s 848us/step - loss: 0.3135 - accuracy: 0.8595 - val_loss: 0.3013 - val_accuracy: 0.8664
Epoch 16/25
10938/10938 [==============================] - 9s 834us/step - loss: 0.3130 - accuracy: 0.8599 - val_loss: 0.3028 - val_accuracy: 0.8660
Epoch 17/25
10938/10938 [==============================] - 9s 848us/step - loss: 0.3135 - accuracy: 0.8598 - val_loss:

10938/10938 [==============================] - 9s 820us/step - loss: 0.3210 - accuracy: 0.8540 - val_loss: 0.3112 - val_accuracy: 0.8581
Epoch 17/25
10938/10938 [==============================] - 9s 832us/step - loss: 0.3214 - accuracy: 0.8535 - val_loss: 0.3113 - val_accuracy: 0.8585
Epoch 18/25
10938/10938 [==============================] - 9s 822us/step - loss: 0.3213 - accuracy: 0.8538 - val_loss: 0.3130 - val_accuracy: 0.8565
Epoch 19/25
10938/10938 [==============================] - 9s 833us/step - loss: 0.3215 - accuracy: 0.8537 - val_loss: 0.3107 - val_accuracy: 0.8604
Epoch 20/25
10938/10938 [==============================] - 9s 826us/step - loss: 0.3213 - accuracy: 0.8539 - val_loss: 0.3125 - val_accuracy: 0.8586
Epoch 21/25
10938/10938 [==============================] - 9s 836us/step - loss: 0.3211 - accuracy: 0.8537 - val_loss: 0.3108 - val_accuracy: 0.8593
Epoch 22/25
10938/10938 [==============================] - 9s 827us/step - loss: 0.3213 - accuracy: 0.8539 - val_loss:

10938/10938 [==============================] - 9s 826us/step - loss: 0.3134 - accuracy: 0.8596 - val_loss: 0.3030 - val_accuracy: 0.8662
Epoch 22/25
10938/10938 [==============================] - 9s 837us/step - loss: 0.3128 - accuracy: 0.8601 - val_loss: 0.3014 - val_accuracy: 0.8659
Epoch 23/25
10938/10938 [==============================] - 9s 831us/step - loss: 0.3129 - accuracy: 0.8605 - val_loss: 0.3023 - val_accuracy: 0.8643
Epoch 24/25
10938/10938 [==============================] - 9s 830us/step - loss: 0.3129 - accuracy: 0.8599 - val_loss: 0.3032 - val_accuracy: 0.8636
Epoch 25/25
10938/10938 [==============================] - 9s 841us/step - loss: 0.3127 - accuracy: 0.8603 - val_loss: 0.3022 - val_accuracy: 0.8649
Epoch 1/25
10938/10938 [==============================] - 10s 860us/step - loss: 0.3132 - accuracy: 0.8583 - val_loss: 0.2995 - val_accuracy: 0.8670
Epoch 2/25
10938/10938 [==============================] - 9s 842us/step - loss: 0.3093 - accuracy: 0.8604 - val_loss: 

...

Epoch 6/25
10938/10938 [==============================] - 10s 938us/step - loss: 0.3085 - accuracy: 0.8623 - val_loss: 0.2930 - val_accuracy: 0.8716
Epoch 7/25
10938/10938 [==============================] - 10s 941us/step - loss: 0.3072 - accuracy: 0.8631 - val_loss: 0.2929 - val_accuracy: 0.8704
Epoch 8/25
10938/10938 [==============================] - 10s 934us/step - loss: 0.3077 - accuracy: 0.8630 - val_loss: 0.2926 - val_accuracy: 0.8693
Epoch 9/25
10938/10938 [==============================] - 10s 938us/step - loss: 0.3073 - accuracy: 0.8634 - val_loss: 0.2922 - val_accuracy: 0.8703
Epoch 10/25
10938/10938 [==============================] - 10s 938us/step - loss: 0.3066 - accuracy: 0.8635 - val_loss: 0.2917 - val_accuracy: 0.8717
Epoch 11/25
10938/10938 [==============================] - 10s 929us/step - loss: 0.3060 - accuracy: 0.8637 - val_loss: 0.2904 - val_accuracy: 0.8719
Epoch 12/25
10938/10938 [==============================] - 10s 939us/step - loss: 0.3065 - accuracy: 0.8

Epoch 11/25
10938/10938 [==============================] - 10s 927us/step - loss: 0.3062 - accuracy: 0.8639 - val_loss: 0.2938 - val_accuracy: 0.8705
Epoch 12/25
10938/10938 [==============================] - 10s 939us/step - loss: 0.3058 - accuracy: 0.8642 - val_loss: 0.2972 - val_accuracy: 0.8669
Epoch 13/25
10938/10938 [==============================] - 10s 934us/step - loss: 0.3053 - accuracy: 0.8646 - val_loss: 0.2947 - val_accuracy: 0.8692
Epoch 14/25
10938/10938 [==============================] - 10s 931us/step - loss: 0.3051 - accuracy: 0.8644 - val_loss: 0.2940 - val_accuracy: 0.8702
Epoch 15/25
10938/10938 [==============================] - 10s 935us/step - loss: 0.3050 - accuracy: 0.8646 - val_loss: 0.2938 - val_accuracy: 0.8722
Epoch 16/25
10938/10938 [==============================] - 10s 936us/step - loss: 0.3047 - accuracy: 0.8648 - val_loss: 0.2937 - val_accuracy: 0.8703
Epoch 17/25
10938/10938 [==============================] - 10s 931us/step - loss: 0.3047 - accuracy:

Epoch 16/25
10938/10938 [==============================] - 9s 816us/step - loss: 0.2903 - accuracy: 0.8683 - val_loss: 0.2818 - val_accuracy: 0.8725
Epoch 17/25
10938/10938 [==============================] - 9s 815us/step - loss: 0.2907 - accuracy: 0.8685 - val_loss: 0.2776 - val_accuracy: 0.8754
Epoch 18/25
10938/10938 [==============================] - 9s 801us/step - loss: 0.2909 - accuracy: 0.8680 - val_loss: 0.2789 - val_accuracy: 0.8748
Epoch 19/25
10938/10938 [==============================] - 9s 813us/step - loss: 0.2901 - accuracy: 0.8684 - val_loss: 0.2787 - val_accuracy: 0.8759
Epoch 20/25
10938/10938 [==============================] - 9s 801us/step - loss: 0.2903 - accuracy: 0.8684 - val_loss: 0.2778 - val_accuracy: 0.8759
Epoch 21/25
10938/10938 [==============================] - 9s 809us/step - loss: 0.2906 - accuracy: 0.8680 - val_loss: 0.2781 - val_accuracy: 0.8756
Epoch 22/25
10938/10938 [==============================] - 9s 809us/step - loss: 0.2901 - accuracy: 0.8681

10938/10938 [==============================] - 10s 897us/step - loss: 0.2990 - accuracy: 0.8668 - val_loss: 0.2869 - val_accuracy: 0.8742
Epoch 22/25
10938/10938 [==============================] - 10s 884us/step - loss: 0.2986 - accuracy: 0.8675 - val_loss: 0.2852 - val_accuracy: 0.8735
Epoch 23/25
10938/10938 [==============================] - 9s 864us/step - loss: 0.2987 - accuracy: 0.8670 - val_loss: 0.2863 - val_accuracy: 0.8719
Epoch 24/25
10938/10938 [==============================] - 9s 854us/step - loss: 0.2980 - accuracy: 0.8677 - val_loss: 0.2846 - val_accuracy: 0.8745
Epoch 25/25
10938/10938 [==============================] - 11s 983us/step - loss: 0.2985 - accuracy: 0.8674 - val_loss: 0.2891 - val_accuracy: 0.8694
Epoch 1/25
10938/10938 [==============================] - 11s 969us/step - loss: 0.3208 - accuracy: 0.8571 - val_loss: 0.3087 - val_accuracy: 0.8662
Epoch 2/25
10938/10938 [==============================] - 10s 957us/step - loss: 0.3174 - accuracy: 0.8589 - val_lo

10938/10938 [==============================] - 10s 882us/step - loss: 0.3104 - accuracy: 0.8598 - val_loss: 0.2954 - val_accuracy: 0.8666
Epoch 2/25
10938/10938 [==============================] - 10s 879us/step - loss: 0.3060 - accuracy: 0.8621 - val_loss: 0.2921 - val_accuracy: 0.8670
Epoch 3/25
10938/10938 [==============================] - 9s 844us/step - loss: 0.3039 - accuracy: 0.8633 - val_loss: 0.2900 - val_accuracy: 0.8702
Epoch 4/25
10938/10938 [==============================] - 9s 867us/step - loss: 0.3022 - accuracy: 0.8637 - val_loss: 0.2892 - val_accuracy: 0.8707
Epoch 5/25
10938/10938 [==============================] - 9s 826us/step - loss: 0.3016 - accuracy: 0.8642 - val_loss: 0.2885 - val_accuracy: 0.8702
Epoch 6/25
10938/10938 [==============================] - 10s 869us/step - loss: 0.3008 - accuracy: 0.8646 - val_loss: 0.2892 - val_accuracy: 0.8710
Epoch 7/25
10938/10938 [==============================] - 11s 1ms/step - loss: 0.3008 - accuracy: 0.8646 - val_loss: 0.2

## Performing test on the real signal

In [ ]:
def getFeatures(sig: np.ndarray) -> np.ndarray:
    features = np.zeros((1, 3 * NFEATURES))
    
    mfcc_feats = mfcc(sig,
                      SAMPLE_RATE,
                      winlen=FRAME_LENGTH,
                      winstep=FRAME_STEP,
                      numcep=NFEATURES,
                      nfilt=NFILTERS,
                      nfft=NFFT,
                      lowfreq=0,
                      highfreq=None,
                      preemph=PREEMPHASIS_COEF,
                      ceplifter=CEPLIFTER,
                      appendEnergy=APPEND_ENERGY,
                      winfunc=WINDOW_FUNCTION)
    d_mfcc = delta(mfcc_feats, N=2)
    d2_mfcc = delta(d_mfcc, N=2)
    
    dataset_on_sample = np.zeros((mfcc_feats.shape[0], 3 * NFEATURES))
    for j in range(mfcc_feats.shape[0]):
        dataset_on_sample[j][:NFEATURES] = mfcc_feats[j][:]
        dataset_on_sample[j][NFEATURES:2 * NFEATURES] = d_mfcc[j][:]
        dataset_on_sample[j][2 * NFEATURES:3 * NFEATURES] = d2_mfcc[j][:]
        
    mfcc_feat = np.concatenate((features, dataset_on_sample), axis=0)
    mfcc_feat = np.delete(mfcc_feat, 0, axis=0)
    
    return mfcc_feat

In [ ]:
# Get vector combined labels:
def combine_labels(sig: np.ndarray, labels: np.ndarray, frame_length: int, frame_step: int, sample_rate: int) -> np.ndarray:
    time_series_labels_speech = np.zeros(sig.shape[0])

    begin = 0
    end = int(frame_length * sample_rate)
    shift = int(frame_step * sample_rate)
    for i in range(labels.shape[0]):
        if labels[i] > 0.5:
            time_series_labels_speech[begin:end] = 1
        begin = begin + shift
        end = end + shift 

    return time_series_labels_speech

In [ ]:
# Get plot:
def plot_result_VAD(sound: np.ndarray, labels: np.ndarray) -> None:
    plt.figure(num=None, figsize=(16, 9), dpi=80, facecolor='w', edgecolor='k')
    
    plt.plot(sound / max(abs(sound)), color='blue')
    plt.plot(labels, color='red')

In [ ]:
# Read the wav file:
(sample_rate, sig) = wav.read(REAL_SIGNAL)
sig = resampy.resample(x=sig, sr_orig=sample_rate, sr_new=SAMPLE_RATE)

features = getFeatures(sig=sig)
labels_MLPClassifier = model.predict(features)

op_MLPClassifier = combine_labels(sig=sig,
                                  labels=labels_MLPClassifier,
                                  frame_length=FRAME_LENGTH,
                                  frame_step=FRAME_STEP,
                                  sample_rate=SAMPLE_RATE)
plot_result_VAD(sound=sig, labels=op_MLPClassifier)

## Getting DNN inner params

In [ ]:
for layer in model.layers: print(layer.get_config(), layer.get_weights())

## Self prediction

In [ ]:
def ReLU(X: np.ndarray) -> np.ndarray:
    return np.array([max(0, el) for el in X])

In [ ]:
def sigmoid(X: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-X))

In [ ]:
def selfPredict(input_batch: np.ndarray, layers: list) -> np.float32:
    # BatchNormalization layer:
    gamma = layers[0].get_weights()[0]
    beta = layers[0].get_weights()[1]
    mean = layers[0].get_weights()[2]
    std = layers[0].get_weights()[3]
    epsilon = layers[0].get_config()['epsilon']
    
    # Hidden layer #1:
    hiddenLayer_weights, hiddenLayer_bias = list(), list()
    for i in range(1, len(layers) - 1):
        hiddenLayer_weights.append(layers[i].get_weights()[0])
        hiddenLayer_bias.append(layers[i].get_weights()[1])
    
    # Output layer:
    finalLayer_weights = layers[len(layers) - 1].get_weights()[0]
    finalLayer_bias = layers[len(layers) - 1].get_weights()[1]
    
    # Forward propagation:
    batchNormalization_result = gamma * (input_batch - mean) / np.sqrt(std + epsilon) + beta
    hiddenLayer_result = batchNormalization_result
    for i in range(len(hiddenLayer_weights)):
        hiddenLayer_result = ReLU(X=(np.dot(hiddenLayer_result, hiddenLayer_weights[i]) + hiddenLayer_bias[i]))
    finalLayer_result = sigmoid(X=(np.dot(hiddenLayer_result, finalLayer_weights) + finalLayer_bias))
    
    return finalLayer_result

In [ ]:
num_frame = 100
print('Self prediction:', selfPredict(input_batch=features[num_frame], layers=model.layers))
print('TF prediction: ', model.predict(features)[num_frame])

## Models evaluation

In [ ]:
def plot_roc_curve(fpr: list, tpr: list, info: list) -> None:
    plt.figure(figsize=(16, 6))
    for i in range(len(fpr)):
        plt.plot(fpr[i], tpr[i])
    plt.grid()
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(info)
    plt.show()

In [ ]:
def modelsEvalution(filename: str, chunk_size: int) -> None:
    # Load part of data:
    data_chunk = pd.read_csv(filename, nrows=chunk_size)
    data_chunk = data_chunk.to_numpy(dtype=np.float32, copy=False)
    labels_valid = data_chunk[:, 0]
    data_valid = data_chunk[:, 1:]
    
    fpr_list, tpr_list = list(), list()
    info_list = list()
    for version in models_inst:
        # Load models:
        model = loadModel(models_inst[version])
        # Make prediction:
        labels_pred_proba = model.predict(data_valid).ravel()
        
        fpr, tpr, thresholds = metrics.roc_curve(labels_valid, labels_pred_proba)
        fpr_list.append(fpr)
        tpr_list.append(tpr)
        auc_score = metrics.roc_auc_score(labels_valid, labels_pred_proba)
        info_list.append(structure_info[version] + ' Auc score: ' + str(float('{:.3f}'.format(auc_score))))
        
    plot_roc_curve(fpr=fpr_list, tpr=tpr_list, info=info_list)

In [ ]:
modelsEvalution(filename=TRAIN_FILE, chunk_size=CHUNK_SIZE)

## Forming C++ array

### BatchNormalization layer

In [ ]:
print('// ====================================================================================================================== //')
print('// ' + model.layers[0].get_config()['name'] + ' //')
print('// ====================================================================================================================== //')
batchNormalization_gamma_string = 'float batchNormalization_gamma[MFCC_SIZE] = '
for i in range(model.layers[0].get_weights()[0].shape[0]):
    if i == 0:
        batchNormalization_gamma_string = batchNormalization_gamma_string + '{' + str(model.layers[0].get_weights()[0][i]) + ', '
    elif i == model.layers[0].get_weights()[0].shape[0] - 1:
        batchNormalization_gamma_string = batchNormalization_gamma_string + str(model.layers[0].get_weights()[0][i]) + '};'
    else:
        batchNormalization_gamma_string = batchNormalization_gamma_string + str(model.layers[0].get_weights()[0][i]) + ', '
        
print(batchNormalization_gamma_string)
print('// ====================================================================================================================== //')
batchNormalization_beta_string = 'float batchNormalization_beta[MFCC_SIZE] = '
for i in range(model.layers[0].get_weights()[1].shape[0]):
    if i == 0:
        batchNormalization_beta_string = batchNormalization_beta_string + '{' + str(model.layers[0].get_weights()[1][i]) + ', '
    elif i == model.layers[0].get_weights()[1].shape[0] - 1:
        batchNormalization_beta_string = batchNormalization_beta_string + str(model.layers[0].get_weights()[1][i]) + '};'
    else:
        batchNormalization_beta_string = batchNormalization_beta_string + str(model.layers[0].get_weights()[1][i]) + ', '
        
print(batchNormalization_beta_string)
print('// ====================================================================================================================== //')
batchNormalization_mean_string = 'float batchNormalization_mean[MFCC_SIZE] = '
for i in range(model.layers[0].get_weights()[2].shape[0]):
    if i == 0:
        batchNormalization_mean_string = batchNormalization_mean_string + '{' + str(model.layers[0].get_weights()[2][i]) + ', '
    elif i == model.layers[0].get_weights()[2].shape[0] - 1:
        batchNormalization_mean_string = batchNormalization_mean_string + str(model.layers[0].get_weights()[2][i]) + '};'
    else:
        batchNormalization_mean_string = batchNormalization_mean_string + str(model.layers[0].get_weights()[2][i]) + ', '
        
print(batchNormalization_mean_string)
print('// ====================================================================================================================== //')
batchNormalization_std_string = 'float batchNormalization_std[MFCC_SIZE] = '
for i in range(model.layers[0].get_weights()[3].shape[0]):
    if i == 0:
        batchNormalization_std_string = batchNormalization_std_string + '{' + str(model.layers[0].get_weights()[3][i]) + ', '
    elif i == model.layers[0].get_weights()[3].shape[0] - 1:
        batchNormalization_std_string = batchNormalization_std_string + str(model.layers[0].get_weights()[3][i]) + '};'
    else:
        batchNormalization_std_string = batchNormalization_std_string + str(model.layers[0].get_weights()[3][i]) + ', '
        
print(batchNormalization_std_string)
print('// ====================================================================================================================== //')
batchNormalization_epsilon_string = 'float batchNormalization_epsilon = '
batchNormalization_epsilon_string = batchNormalization_epsilon_string + str(model.layers[0].get_config()['epsilon']) + ';'
        
print(batchNormalization_epsilon_string)
print('// ====================================================================================================================== //')

### Hidden dense layers

In [ ]:
num_hiddenLayers = len(model.layers) - 1
for i in range(1, num_hiddenLayers):
    print('// ====================================================================================================================== //')
    print('// ' + model.layers[i].get_config()['name'] + ' //')
    print('// ====================================================================================================================== //')
    if i == 1:
        first_dim = 'MFCC_SIZE'
    else:
        first_dim = 'HIDDEN_NODES_SIZE_' + str(i - 1)
    hiddenLayer_weights_string = 'float HiddenLayer_' + str(i) + '_weights['+ first_dim +'][HIDDEN_NODES_SIZE_' + str(i) + '] = '
    hiddenLayer_bias_string = 'float HiddenLayer_' + str(i) + '_bias[HIDDEN_NODES_SIZE_' + str(i) + '] = '
    hidden_weights = model.layers[i].get_weights()[0]
    hidden_bias = model.layers[i].get_weights()[1]
    for j in range(hidden_weights.shape[0]):
        if j == 0:
            hiddenLayer_weights_string = hiddenLayer_weights_string + '{'
        for k in range(hidden_weights.shape[1]):
            if k == 0:
                hiddenLayer_weights_string = hiddenLayer_weights_string + '{' + str(hidden_weights[j][k]) + ', '
            elif k == hidden_weights.shape[1] - 1:
                if j == hidden_weights.shape[0] - 1:
                    hiddenLayer_weights_string = hiddenLayer_weights_string + str(hidden_weights[j][k]) + '}'
                else:
                    hiddenLayer_weights_string = hiddenLayer_weights_string + str(hidden_weights[j][k]) + '}, '
            else:
                hiddenLayer_weights_string = hiddenLayer_weights_string + str(hidden_weights[j][k]) + ', '
        if j == hidden_weights.shape[0] - 1:
            hiddenLayer_weights_string = hiddenLayer_weights_string + '};'
    for j in range(hidden_bias.shape[0]):
        if j == 0:
            hiddenLayer_bias_string = hiddenLayer_bias_string + '{' + str(hidden_bias[j]) + ', '
        elif j == hidden_bias.shape[0] - 1:
            hiddenLayer_bias_string = hiddenLayer_bias_string + str(hidden_bias[j]) + '};'
        else:
            hiddenLayer_bias_string = hiddenLayer_bias_string + str(hidden_bias[j]) + ', '
    print(hiddenLayer_weights_string)
    print('// ====================================================================================================================== //')
    print(hiddenLayer_bias_string)
    print('// ====================================================================================================================== //')
name_for_output_layer = i

### Output dense layer

In [ ]:
output_weights = model.layers[len(model.layers) - 1].get_weights()[0].reshape(1, -1)[0]
output_bias = model.layers[len(model.layers) - 1].get_weights()[1]
print('// ====================================================================================================================== //')
print('// ' + model.layers[len(model.layers) - 1].get_config()['name'] + ' //')
print('// ====================================================================================================================== //')
outputLayer_weights_string = 'float outputLayer_weights[HIDDEN_NODES_SIZE_' + str(name_for_output_layer) + '] = '
for i in range(output_weights.shape[0]):
    if i == 0:
        outputLayer_weights_string = outputLayer_weights_string + '{' + str(output_weights[i]) + ', '
    elif i == model.layers[len(model.layers) - 1].get_weights()[0].shape[0] - 1:
        outputLayer_weights_string = outputLayer_weights_string + str(output_weights[i]) + '};'
    else:
        outputLayer_weights_string = outputLayer_weights_string + str(output_weights[i]) + ', '
        
print(outputLayer_weights_string)
print('// ====================================================================================================================== //')
outputLayer_bias_string = 'float outputLayer_bias = '
outputLayer_bias_string = outputLayer_bias_string + str(output_bias[0]) + ';'
        
print(outputLayer_bias_string)
print('// ====================================================================================================================== //')

### Features frame

In [ ]:
features_frame_string = 'float test_input[MFCC_SIZE] = '
for i in range(features[num_frame].shape[0]):
    if i == 0:
        features_frame_string = features_frame_string + '{' + str(features[num_frame][i]) + ', '
    elif i == features[num_frame].shape[0] - 1:
        features_frame_string = features_frame_string + str(features[num_frame][i]) + '};'
    else:
        features_frame_string = features_frame_string + str(features[num_frame][i]) + ', '
        
print(features_frame_string)